# dropping neccesasary libs

In [1]:
import requests
import mechanize
from bs4 import BeautifulSoup as bs
from urllib.request import urlopen, urljoin, Request

# attempt 1(html page code not fully displayed)

In [8]:
headers = {'accept': '*/*',
         'user-agent': 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Ubuntu Chromium/73.0.3683.86 Chrome/73.0.3683.86 Safari/537.36'}
base_url = 'https://www.sciencedirect.com/journal/computer-speech-and-language/vol/59/suppl/C'
def lul_parser(base_url,headers):
    session = requests.Session()
    request = session.get(base_url,headers = headers)
    if request.status_code == 200:
        soup = bs(request.content, 'html.parser')
        a = soup.find('a', attrs = "anchor article-content-title u-margin-xs-top u-margin-s-bottom")
        #href = soup.find_all('a', attrs = 'anchor js-issue-item-link text-m')['href']
        print(a)
        #print(soup)
    else:
        print('shit')

# attempt n 2 : just copied html code from necessary source(url) and get links

In [12]:
outF = open('main_page_html.txt', "r")
main_page_html = outF.readlines()

In [ ]:
main_soup = bs(main_page_html)
a_list = main_soup.find_all('a')
ssilki = []
for i in a_list:
    g = str(i).replace('<a class="anchor js-issue-item-link text-m" href="','')
    ssilki.append('https://www.sciencedirect.com'+g.split('"><')[0])

In [489]:
len(ssilki),ssilki[0]

(89,
 'https://www.sciencedirect.com/journal/computer-speech-and-language/vol/59/suppl/C')

In [81]:
def lul_parser(base_url,headers):
    session = requests.Session()
    request = session.get(base_url,headers = headers)
    if request.status_code == 200:
        soup = bs(request.content, 'html.parser')
        divs = soup.find_all('div', attrs = 'issue-item u-margin-s-bottom')
        
        #href = soup.find_all('a', attrs = 'anchor js-issue-item-link text-m')['href']
        print(divs)
    else:
        print('shit')

In [63]:
print(('Main page have {} links of volumes from 2001-2019').format(len(ssilki)))

Main page have 89 links of volumes from 2001-2019


# target = 'date keywords title abstract text author journal'

# attempt number 3,and its working only fo volume page, main page suck
### main page of  journal have links(89) for volumes(every release),  we need to get this links and in every links we have issues(5-20). From this issues's links we can download our target objects 

## a function that returns links to articles from journal issues 

In [9]:
def links_from_volumes(links_from_main_page):

    br = mechanize.Browser()
    br.set_handle_robots(False)
    br.addheaders = [('User-agent', 'Mozilla/5.0 (X11; U; Linux i686; en-US; rv:1.9.0.1) Gecko/2008071615 Fedora/3.0.1-1.fc9 Firefox/3.0.1')]
    
    l1nks = []
    
    for i in links_from_main_page:
        html = br.open(i,timeout=100).read()
        soup = bs(html,'html5lib')
        links = soup.find_all('a', {"class":'anchor article-content-title u-margin-xs-top u-margin-s-bottom'})

        for j in links:
            l1nks.append('https://www.sciencedirect.com'+str(j).replace('<a class="anchor article-content-title u-margin-xs-top u-margin-s-bottom" href="','').split('" id=')[0])
            
    return l1nks        

In [72]:
print('четвертая ссылка из массива со ссылками: {}'.format(ssilki[3]))

четвертая ссылка из массива со ссылками: https://www.sciencedirect.com/journal/computer-speech-and-language/vol/56/suppl/C


In [98]:
final_links = links_from_volumes(ssilki)

## we have All  links(880), now we gonna download target objects 

In [ ]:
g = open('final_links.txt','a+')
for i in final_links:
    
    g.write('\r\n'+i)

In [14]:
br = mechanize.Browser()
br.set_handle_robots(False)
br.addheaders = [('User-agent', 'Mozilla/5.0 (X11; U; Linux i686; en-US; rv:1.9.0.1) Gecko/2008071615 Fedora/3.0.1-1.fc9 Firefox/3.0.1')]

In [21]:
def final_parser(links):
    #upperAlphabet = ['Q','W','E','R','T','Y','U','O','P','A','S','D','F','G','H','J','K','L','Z','X','C','V','B','N','M']
    target = []
    errors = []

    for k in links:
        html = br.open(k,timeout=200).read()
        soup = bs(html,'html5lib')
        
        #*******************DATE**************************
        
        date = (soup.find('div',{'class':'text-xs'},{"title":'Go to table of contents for this volume/issue'}).text).split(', ')
        for i in date:
            if ('January' in i) or ('February' in i) or ('March' in i) or ('April' in i) or ('May' in i) or ('June' in i) or ('July' in i) or ('August' in i) or ('September' in i) or ('October' in i) or ('November' in i) or ('December' in i):
                date = i
                
        #*******************KEYWORDS**********************
        
        keywordz =[]
        keywords = soup.find('div',{'class':'Keywords u-font-serif'})
        if type(keywords) == type(None):
            errors.append('KeywordNone'+k)
        else:
            for i in keywords.find_all('div',{'class':'keyword'}):
                keywordz.append(i.text)
        #*******************TITLE*************************
        
        title = soup.find('h1',{'class':'Head u-font-serif u-h2 u-margin-s-ver'}).text
        
        #*******************ABSTRACT**********************
        
        abstract = soup.find('div',{'class':'abstract author'})
        if type(abstract)==type(None):
            errors.append('AbstractNone'+k)
        else:
            abstract = (abstract.text).replace('Abstract','')
                             
        #*******************JOURNAL***********************
        #in this case,everywhere we have same journal
        #*******************AUTHOR************************
        authors = []                     
        author = soup.find('div',{'class':'author-group'})
        given_name = author.find_all('span',{'class':'text given-name'})
        surname = author.find_all('span',{'class':'text surname'})
        for i in range(len(given_name)):
            authors.append(given_name[i].text+' '+surname[i].text)
        
                             
        target.append({ 'Date':date,
                        'Title':title,
                        'Keywords':keywordz,
                        'Abstract': abstract,
                        'Jornal':'Computer Speech & Language',
                        'Author': authors})
    return target        #,errors

In [470]:
final_data = final_parser(final_links)

In [500]:
final_data.to_csv('Computer Speech & Language.csv')

In [14]:
df = pandas.read_csv('Computer Speech & Language.csv')

In [15]:
df = df.dropna()

In [16]:
df.to_csv('final_data.csv')

## парсим журналы с 1986-2001

In [16]:
outF = open('old_page_html.txt', "r")
old_page_html = outF.readlines()

In [ ]:
old_soup = bs(old_page_html)
old_list = old_soup.find_all('a')
old_ssilki = []
for i in old_list:
    g = str(i).replace('<a class="anchor js-issue-item-link text-m" href="','')
    old_ssilki.append('https://www.sciencedirect.com'+g.split('"><')[0])

In [8]:
len(old_ssilki)

53

## get links of issues from 53 volumes

In [10]:
older_links = links_from_volumes(old_ssilki)
final_older_data = final_parser(older_links)
older_final_data = pandas.DataFrame(final_older_data)
older_final_data = older_final_data.dropna()
older_final_data.to_csv('final_data_older.csv')
QWERTY = pandas.read_csv('final_data.csv')

In [43]:
result = pandas.concat([QWERTY, older_final_data], ignore_index=True)

/home/beksultan/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  """Entry point for launching an IPython kernel.


In [48]:
result.to_csv('result.csv')